In [1]:
import os
from glob import glob
import torch
import torch.nn as nn
from torchvision import models as M
import time
import csv
from PIL import Image
from tqdm.notebook import tqdm
from models import Effnet, Swin_trm
from args import build_opt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from loaders import build_loader, build_loader_new
## args-->opt
opt = build_opt()

torch.manual_seed(42)
np.random.seed(42)
opt.model_name = 'efficientnet_b6'
opt.batch_size = 32

...........................................................................
...param: traindir: ../train/labeled_data/
...param: train_metadir: ../
...param: train_metafile: train_label.csv
...param: testdir: ../test
...param: test_metadir: None
...param: test_metafile: None
...param: epochs: 10
...param: num_workers: 4
...param: batch_size: 64
...param: lr: 0.1
...param: weight_decay: 1e-05
...param: num_classes: 2
...param: step_size: 5
...param: gamma: 0.9
...param: ckpt_name: None
...param: submission_name: submission.csv
...param: model_name: None
...param: kfold: 5
...param: device: cuda
...param: resultdir: ./result
...........................................................................


In [2]:
opt.model_name = 'swin_base_patch4_window12_384_in22k'

In [3]:
## model
## 加载模型
model = Swin_trm(model_name=opt.model_name)
## load ckpts
ckpts = torch.load('../ckpts/8_30/model_best.pth')
model_ckpts = ckpts['model']
model.load_state_dict(model_ckpts)
model = model.cuda()
print('This model‘s running loss is: {}'.format(ckpts['loss']))

/root/miniconda3/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


This model‘s running loss is: 0.36087887206185854


In [2]:
## model Effient net
## 加载模型
model = Effnet(model_name=opt.model_name)
## load ckpts
ckpts = torch.load('../ckpts/8_29/model_best.pth')
# model_ckpts = ckpts['model']
model.load_state_dict(ckpts)
model = model.cuda()
# print('This model‘s running loss is: {}'.format(ckpts['loss']))

In [4]:
opt.resultdir = './result/901/'

In [5]:
## inference
test_loader = build_loader(
    imagedir=opt.testdir,
    batch_size=opt.batch_size,
    num_workers=opt.num_workers,
    metadir=opt.test_metadir,
    metafile=opt.test_metafile,
    require_label=False
)

print('...start predicting')

model.eval()
to_prob = nn.Softmax(dim=1)
with torch.no_grad():
    imagenames, probs = list(), list()
    for batch_idx, batch in tqdm(enumerate(test_loader), total=len(test_loader)):
        image, imagename = batch
        image = image.to(opt.device)
        pred = model(image)
        prob = to_prob(pred)
        prob = list(prob.data.cpu().numpy())
        imagenames += imagename
        probs += prob

with open(os.path.join(opt.resultdir, 'submission.csv'), 'w', encoding='utf8') as fp:
    writer = csv.writer(fp)
    writer.writerow(['imagename', 'defect_prob'])
    for info in zip(imagenames, probs):
        imagename, prob = info
        writer.writerow([imagename, str(prob[1])])

...start predicting


  0%|          | 0/125 [00:00<?, ?it/s]

In [5]:
opt.resultdir

'./result'

In [6]:
## test
submit_df =pd.read_csv('./result/901/submission.csv')
submit_df.shape[0]

4000